In [1]:
import sys
#sys.path.insert(1, '/home/user/Research/base-WingWatch')
sys.path.insert(1, '/home/main/Research/WingWatch')

from WingWatch.Equipment import station
from WingWatch.Equipment import antenna
from WingWatch.Intersections import montecarlo
from WingWatch.Tools import translation
from WingWatch.Calibration import clean_data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
cleaned_gps_data = clean_data.clean_gps_data_csv("track_points.csv")


/home/main/Research/WingWatch/WingWatch/Calibration/clean_data.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gps_reduced_col['time'] = pd.to_datetime(df_gps_reduced_col['time']).astype(int) / 10**9


In [3]:
cleaned_tag_data = clean_data.clean_motus_Tag_data("allTagsSub390_1.csv",start_date="",end_date='2025-06-30',calibration_tags_list=[82521,82516])


/home/main/Research/WingWatch/WingWatch/Calibration/clean_data.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calibration_tags_only['port'] = port_cleaned


In [4]:
SOI = ['SE_Light_434','Black_Rock_434','BIWF_Turbine1']
SOI_Lat = [41.15308,41.14792,41.12579]
SOI_Long = [-71.55141,-71.59009,-71.50742]
SOI_Alt = [20,20,20]

In [5]:
data_dataframe = clean_data.merge_gps_and_tag_data(cleaned_tag_data,cleaned_gps_data,stations_of_interest=SOI)
#I left an extra tag on during the calibration runs... the below removes it. 
data_dataframe = data_dataframe[~((data_dataframe['mfgID'] == '61332A2D') & (data_dataframe['tsCorrected'] > 1717214399))]


In [6]:
for i in range(len(SOI)):
    for j in range(5):
        temp_data_frame = clean_data.generate_antenna_station_calib_file(data_dataframe,station_of_interest=SOI[i],antenna=j+1,ref_lat = SOI_Lat[i], ref_long = SOI_Long[i], ref_alt = SOI_Alt[i])
        temp_data_frame = temp_data_frame.dropna(axis=0)
        plt.scatter(temp_data_frame.X,temp_data_frame.Y,c=temp_data_frame.RSSI)
        plt.scatter(0,0,color='black')
        plt.savefig(SOI[i] + '_' + str(j) + '_calib_fig.pdf')
        plt.close()
        temp_data_frame.to_csv(SOI[i] + '_' + str(j) + '_calib.csv')

In [7]:
SE_Light = data_dataframe[data_dataframe['recvDeployName'] =='SE_Light_434']

In [8]:
SE_Light[SE_Light.mfgID=='61332A2D']

,tsCorrected,sig,port,recvDeployName,motusTagID,mfgID,Y,X,ele
0,1.710948e+09,-89.0,2,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
1,1.710948e+09,-95.0,3,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
2,1.710948e+09,-90.0,5,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
3,1.710948e+09,-95.0,3,SE_Light_434,82521,61332A2D,41.153206,-71.551734,47.0
4,1.710948e+09,-91.0,5,SE_Light_434,82521,61332A2D,41.153206,-71.551734,47.0
...,...,...,...,...,...,...,...,...,...
51547,1.717171e+09,-107.0,1,SE_Light_434,82521,61332A2D,41.153145,-71.551676,60.6
51548,1.717171e+09,-96.0,3,SE_Light_434,82521,61332A2D,41.153143,-71.551677,60.6
51549,1.717171e+09,-78.0,2,SE_Light_434,82521,61332A2D,41.153143,-71.551677,60.6
51550,1.717171e+09,-81.0,5,SE_Light_434,82521,61332A2D,41.153143,-71.551677,60.6


In [9]:
BRR = data_dataframe[data_dataframe['recvDeployName'] =='Black_Rock_434']

In [10]:
filtered_df = BRR[~((BRR['mfgID'] == '61332A2D') & (BRR['tsCorrected'] > 1717214399))]


In [11]:
filtered_df.tsCorrected[98695]

1717603050.0

In [12]:
BRR = data_dataframe[data_dataframe['recvDeployName'] =='SE_Light_434']

In [13]:
data_dataframe

,tsCorrected,sig,port,recvDeployName,motusTagID,mfgID,Y,X,ele
0,1.710948e+09,-89.0,2,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
1,1.710948e+09,-95.0,3,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
2,1.710948e+09,-90.0,5,SE_Light_434,82521,61332A2D,41.153206,-71.551734,51.8
3,1.710948e+09,-95.0,3,SE_Light_434,82521,61332A2D,41.153206,-71.551734,47.0
4,1.710948e+09,-91.0,5,SE_Light_434,82521,61332A2D,41.153206,-71.551734,47.0
...,...,...,...,...,...,...,...,...,...
98691,1.717603e+09,-78.0,2,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98692,1.717603e+09,-80.0,3,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98693,1.717603e+09,-71.0,4,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98694,1.717603e+09,-76.0,5,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2


In [14]:
filtered_df

,tsCorrected,sig,port,recvDeployName,motusTagID,mfgID,Y,X,ele
988,1.710948e+09,-109.0,2,Black_Rock_434,82521,61332A2D,41.152766,-71.557819,70.4
989,1.710948e+09,-98.0,1,Black_Rock_434,82521,61332A2D,41.152766,-71.557819,70.4
996,1.710948e+09,-109.0,2,Black_Rock_434,82521,61332A2D,41.152709,-71.558093,71.8
997,1.710948e+09,-98.0,1,Black_Rock_434,82521,61332A2D,41.152709,-71.558093,71.8
1001,1.710948e+09,-107.0,2,Black_Rock_434,82521,61332A2D,41.152690,-71.558183,73.8
...,...,...,...,...,...,...,...,...,...
98691,1.717603e+09,-78.0,2,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98692,1.717603e+09,-80.0,3,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98693,1.717603e+09,-71.0,4,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2
98694,1.717603e+09,-76.0,5,Black_Rock_434,82516,4C782D07,41.147889,-71.589883,24.2


In [15]:
1717692431-1717545600

146831

In [16]:
max(cleaned_tag_data.tsCorrected)

1717604230.0

In [17]:
1717172089.0/1717545600

0.9997825321202535